In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X, y = make_classification(
    n_features=10, 
    n_samples=1000, 
    n_informative=4,
    n_redundant=6,
    n_repeated=0,
    n_classes=2, 
    random_state=42
)

In [3]:
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

model = DecisionTreeClassifier(criterion="entropy", max_depth=10) # criteria: "gini" or "entropy", max_depth=5 or 10
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.89      0.85      0.87       122
           1       0.86      0.90      0.88       128

    accuracy                           0.88       250
   macro avg       0.88      0.88      0.88       250
weighted avg       0.88      0.88      0.88       250



In [6]:
from sklearn.model_selection import cross_val_score

cross_val_score(DecisionTreeClassifier(criterion="gini", max_depth=5), X, y, cv=5)

array([0.865, 0.86 , 0.895, 0.865, 0.89 ])

In [8]:
cross_val_score(DecisionTreeClassifier(criterion="entropy", max_depth=5), X, y, cv=5)

array([0.84 , 0.855, 0.885, 0.865, 0.84 ])

In [10]:
criterion = ["gini", "entropy"]
max_depth = [5, 10, 15]

avg_scores = {}

for c in criterion:
    for d in max_depth:
        clf = DecisionTreeClassifier(criterion=c, max_depth=d)
        score_list = cross_val_score(clf, X, y, cv=5)
        avg_scores[c + "_" + str(d)] = np.average(score_list)
        
avg_scores

{'gini_5': 0.876,
 'gini_10': 0.8799999999999999,
 'gini_15': 0.877,
 'entropy_5': 0.86,
 'entropy_10': 0.8740000000000002,
 'entropy_15': 0.8710000000000001}

In [12]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(
    DecisionTreeClassifier(),
    {'criterion': ["gini", "entropy"],'max_depth': [5, 10, 15]},
    cv=5,
    return_train_score=False
)
clf.fit(X, y)
clf.cv_results_

{'mean_fit_time': array([0.02312665, 0.01985769, 0.01544156, 0.01535716, 0.02054477,
        0.02214661]),
 'std_fit_time': array([0.01373449, 0.00545487, 0.00219276, 0.00214485, 0.00149847,
        0.00037904]),
 'mean_score_time': array([0.0034688 , 0.00140157, 0.00174665, 0.00160456, 0.00119834,
        0.00178938]),
 'std_score_time': array([0.00238238, 0.00049301, 0.00040652, 0.0004947 , 0.00039921,
        0.00039519]),
 'param_criterion': masked_array(data=['gini', 'gini', 'gini', 'entropy', 'entropy',
                    'entropy'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[5, 10, 15, 5, 10, 15],
              mask=[False, False, False, False, False, False],
        fill_value=999999),
 'params': [{'criterion': 'gini', 'max_depth': 5},
  {'criterion': 'gini', 'max_depth': 10},
  {'criterion': 'gini', 'max_depth': 15},
  {'criterion': 'entropy', 'max_depth': 5},
  {'cr

In [16]:
import pandas as pd
df = pd.DataFrame(clf.cv_results_)
df.head(3)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.023127,0.013734,0.003469,0.002382,gini,5,"{'criterion': 'gini', 'max_depth': 5}",0.865,0.865,0.895,0.865,0.890,0.876,0.013565,3
1,0.019858,0.005455,0.001402,0.000493,gini,10,"{'criterion': 'gini', 'max_depth': 10}",0.875,0.885,0.870,0.880,0.895,0.881,0.008602,2
2,0.015442,0.002193,0.001747,0.000407,gini,15,"{'criterion': 'gini', 'max_depth': 15}",0.850,0.875,0.870,0.910,0.905,0.882,0.022494,1


In [18]:
df[["param_criterion", "param_max_depth", "mean_test_score"]]

,param_criterion,param_max_depth,mean_test_score
0,gini,5,0.876
1,gini,10,0.881
2,gini,15,0.882
3,entropy,5,0.857
4,entropy,10,0.874
5,entropy,15,0.869


In [20]:
clf.best_params_

{'criterion': 'gini', 'max_depth': 15}

In [22]:
model = clf.best_estimator_
model

DecisionTreeClassifier(max_depth=15)

In [24]:
# Now try with different model with different parameter
from sklearn import svm

model_params = {
    'decision_tree': {
        'model': DecisionTreeClassifier(),
        'params' : {
            'criterion': ["gini", "entropy"],
            'max_depth': [5, 10, 15]
        }  
    },
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']            
        }
    }
}

scores = []

for key, val in model_params.items():
    clf = GridSearchCV(
        val['model'],
        val['params'],
        cv=5,
        return_train_score=False
    )
    clf.fit(X, y)
    scores.append({
        'model': key,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
scores

[{'model': 'decision_tree',
  'best_score': 0.8790000000000001,
  'best_params': {'criterion': 'gini', 'max_depth': 15}},
 {'model': 'svm',
  'best_score': 0.9359999999999999,
  'best_params': {'C': 10, 'kernel': 'rbf'}}]

In [26]:

df = pd.DataFrame(scores, columns=["model", "best_score", "best_params"])
df

,model,best_score,best_params
0,decision_tree,0.879,"{'criterion': 'gini', 'max_depth': 15}"
1,svm,0.936,"{'C': 10, 'kernel': 'rbf'}"
